In [10]:
import sys
sys.path.append('../')

import stablemotifs
import biolqm
import ginsim
import StableMotifs as sm
import PyBoolNet
import boolsim
import timeit
import timeout_decorator
from timeout_decorator import TimeoutError
import pandas as pd
import glob
from colomoto_jupyter import tabulate
import cabean
import pypint

## Parameters of the bechmark

In [2]:

#the folder in which all models we whish to run are saved in the booleanned format with .booleannet extention. 
models_path='models_for_benchmark/'

#run_ens_size is the number of times each method is repeatedly run IN THE SAME ITERATION. 
#this means that the timeout has to be increased proporionally to account for all runs
#In the final output the minimum (best) runtime is saved out of all in the ensemble 
run_ens_size=1 

#timeout limit in seconds for each method (all runs within the run_ens_size range). 
#If the timeout limit is reached the output will be -1
timeout_limit=3600 


#name of methods to be exluded from the run. This can be set manually if we know 
#that some methods are not relevant or take too much time
exclude_from_run=set(['PyBoolNet'])
#exclude_from_run=set(['bioLQM','PyBoolNet_Asp','PyBoolNet','Pint']) 
#uncomment the line abovr if you want only methods that look for complex attractors

#the excel file in which the dataframe summarizing the results will be saved.
output_file_name='timing_results.xlsx'


In [3]:
@timeout_decorator.timeout(timeout_limit)
def run_timed_test(method_runstring, setup, repeat):

    return timeit.repeat(method_runstring,setup=setup, repeat=repeat, number=1)

try: #if the file already exists we dont overwrite it
    result_df=pd.read_excel(output_file_name)
    result_df=result_df.reset_index(drop=True)
except FileNotFoundError:
    result_df=pd.DataFrame()
    
#grabbing all the .booleannet models from the folder
model_set=glob.glob(models_path+'*.booleannet')
#model_set=['PSO.txt', 'phase_switch.txt','myeloid.txt'] 

for model_file_name in model_set:
    print('Running model %s'%model_file_name)
    
    base_setup_string='''import stablemotifs
import biolqm
import ginsim
import StableMotifs as sm
import PyBoolNet
import boolsim
import cabean
import pypint
from colomoto_jupyter import tabulate


model_file_name='%s'
with open(model_file_name) as f:
    rules=f.read().replace('#BOOLEAN RULES','').strip()
biolqm_model = biolqm.load(model_file_name, "booleannet")
pypint_model=biolqm.to_pint(biolqm_model)'''%model_file_name

    method_runstrings={'sm_jgtz': "a=stablemotifs.load(biolqm_model, quiet=True).attractors\ntabulate(a).to_excel('attr_output/attr_output_sm_jgtz_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')",
                   
                   
 'stable_motifs_new': "ar = sm.AttractorRepertoire.from_primes(primes, max_simulate_size=max_simulate_size)\nex.attractor_dataframe(ar).to_excel('attr_output/attr_output_stable_motifs_new_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')",
                   
                   
 'PyBoolNet': 'PyBoolNet.Attractors.compute_attractors_tarjan(PyBoolNet.StateTransitionGraphs.primes2stg(primes, "asynchronous"))\ntabulate(a).to_excel(\'attr_output/attr_output_PyBoolNet_\'+model_file_name.split(\'.\')[0].split(\'/\')[-1]+\'.xlsx\')',
                   
                   
 'boolsim': "a=boolsim.attractors(biolqm_model)\ntabulate(a).to_excel('attr_output/attr_output_boolsim_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')",
                   
                   
 'bioLQM': "a=biolqm.fixpoints(biolqm_model)\ntabulate(a).to_excel('attr_output/attr_output_bioLQM_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')",
                   
                   
 'PyBoolNet_Asp': "a=PyBoolNet.AspSolver.steady_states(primes)\ntabulate(a).to_excel('attr_output/attr_output_PyBoolNet_Asp_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')",
                   
                   
 'CABEAN': "a=cabean.attractors(biolqm_model)\ntabulate(a).to_excel('attr_output/attr_output_CABEAN_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')",
                   
                   
 'Pint': "a=pypint.fixpoints(pypint_model)\ntabulate(a).to_excel('attr_output/attr_output_Pint_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')"}

    method_setups={'sm_jgtz':base_setup_string,

    'stable_motifs_new':base_setup_string+'''
max_simulate_size=20
rules_pbn = sm.Format.booleannet2bnet(rules)
primes = PyBoolNet.FileExchange.bnet2primes(rules_pbn)
import StableMotifs.Export as ex
''',

    'PyBoolNet':base_setup_string+'''
rules_pbn = sm.Format.booleannet2bnet(rules)
primes = PyBoolNet.FileExchange.bnet2primes(rules_pbn)
''',

    'boolsim':base_setup_string,

    'bioLQM':base_setup_string,

    'PyBoolNet_Asp':base_setup_string+'''
rules_pbn = sm.Format.booleannet2bnet(rules)
primes = PyBoolNet.FileExchange.bnet2primes(rules_pbn)
''',

    'CABEAN':base_setup_string,

    'Pint':base_setup_string}

    method_runtimes={i:[] for i in method_runstrings}
    
    
    for method_name in set(method_runstrings)-exclude_from_run:
        print('method:',method_name)
        try:
            method_runtimes[method_name]=run_timed_test(method_runstrings[method_name], setup=method_setups[method_name], repeat=run_ens_size)
        except Exception as e:
            print(str(e))
            method_runtimes[method_name].append(-1)
    runtime_minimums={k:min(method_runtimes[k]) for k in method_runtimes.keys()-exclude_from_run} 
    runtime_minimums['model']=model_file_name.split('.')[0].split('/')[-1]
    result_df=result_df.append([runtime_minimums],ignore_index=True)
    result_df=result_df.reset_index(drop=True)

    result_df.to_excel('timing_results.xlsx')

Running model models_for_benchmark/cell_cycle_2016.booleannet
method: Pint
[Errno 2] No such file or directory: 'attr_output/attr_output_Pint_cell_cycle_2016.xlsx'
method: boolsim
[Errno 2] No such file or directory: 'attr_output/attr_output_boolsim_cell_cycle_2016.xlsx'
method: sm_jgtz
[Errno 2] No such file or directory: 'attr_output/attr_output_sm_jgtz_cell_cycle_2016.xlsx'
method: stable_motifs_new
[Errno 2] No such file or directory: 'attr_output/attr_output_stable_motifs_new_cell_cycle_2016.xlsx'
method: PyBoolNet_Asp
[Errno 2] No such file or directory: 'attr_output/attr_output_PyBoolNet_Asp_cell_cycle_2016.xlsx'
method: bioLQM
[Errno 2] No such file or directory: 'attr_output/attr_output_bioLQM_cell_cycle_2016.xlsx'
method: CABEAN
Command 'cabean -asynbn -steadystates -newtarjan -newpred -compositional 2 /tmp/cabeandfwireni.ispl' returned non-zero exit status -11
Command line: cabean -asynbn -steadystates -newtarjan -newpred -compositional 2 /tmp/cabeandfwireni.ispl

Running mo

In [4]:
result_df.to_excel(output_file_name)

In [11]:

import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(10,8))
g=sns.barplot(list(runtime_minimums.keys()), list(runtime_minimums.values()),log=True)
for k, v in enumerate(runtime_minimums.values()):
    g.text(k,v, round(v,3), color='black', ha="center")
plt.xlabel('Method', fontsize=14)
plt.ylabel('Best time', fontsize=14)
plt.title('model:'+model_file_name.replace('.txt','')+'; run_ens:%d'%run_ens_size,fontsize=16)
plt.grid()
plt.savefig('benchmark_result_'+model_file_name.replace('.txt','.png'))


ModuleNotFoundError: No module named 'seaborn'

In [6]:
result_df

,Unnamed: 0,boolsim,PyBoolNet_Asp,CABEAN,stable_motifs_new,Pint,bioLQM,sm_jgtz,model
0,0.0,-1,-1,-1,-1,-1,-1,-1,cell_cycle_2016
1,1.0,-1,-1,-1,-1,-1,-1,-1,2161_Guard_Cell_Abscisic_Acid_Signaling
2,NaN,-1,-1,-1,-1,-1,-1,-1,cell_cycle_2016
3,NaN,-1,-1,-1,-1,-1,-1,-1,2161_Guard_Cell_Abscisic_Acid_Signaling
